# DNSMASQ

## DNSMASQ Stichpunkten

- DNSMASQ stellt einen DNS und DHCP Serverdienst zur Verfügung
- Erwartet ein statisch konfiguriertes Netzwerk auf dem Server

## Voraussetzungen Server

- Paket dnsmasq
- Einträge /etc/network/interfaces
- Richtige /etc/hosts auf DNSMASQ-Server
- Ggf. /etc/resolv.conf

----------

## Voraussetzung Client

- Einträge in /etc/network/interfaces
- Link /etc/resolv.conf auf /etc/reslov.conf.orig umbenennen
- Datei /etc/resolv.conf erstellen

-------

## Server Installation

In [1]:
apt policy dnsmasq*

dnsmasq:
  Installiert:           2.75-1ubuntu0.16.04.4
  Installationskandidat: 2.75-1ubuntu0.16.04.4
  Versionstabelle:
 *** 2.75-1ubuntu0.16.04.4 500
        500 http://us.archive.ubuntu.com/ubuntu xenial-updates/universe amd64 Packages
        500 http://us.archive.ubuntu.com/ubuntu xenial-updates/universe i386 Packages
        100 /var/lib/dpkg/status
     2.75-1ubuntu0.16.04.3 500
        500 http://security.ubuntu.com/ubuntu xenial-security/universe amd64 Packages
        500 http://security.ubuntu.com/ubuntu xenial-security/universe i386 Packages
     2.75-1 500
        500 http://us.archive.ubuntu.com/ubuntu xenial/universe amd64 Packages
        500 http://us.archive.ubuntu.com/ubuntu xenial/universe i386 Packages
dnsmasq-base:
  Installiert:           2.75-1ubuntu0.16.04.4
  Installationskandidat: 2.75-1ubuntu0.16.04.4
  Versionstabelle:
 *** 2.75-1ubuntu0.16.04.4 500
        500 http://us.archive.ubuntu.com/ubuntu xenial-updates/main amd64 Packages
        100 /var/lib/dpkg

limit_output extension: Maximum message size of 1000 exceeded with 1008 characters

In [2]:
cd /etc

In [3]:
sudo cp dnsmasq.conf dnsmasq.conf.orig

In [4]:
grep -v '^#' /etc/dnsmasq.conf.orig | grep -v '^$' | sudo tee dnsmasq.conf
# Alternativ: grep -v '^#' /etc/dnsmasq.conf.orig | sudo cat -s > dnsmasq.conf)

In [10]:
cat dnsmasq.conf

In [5]:
sudo systemctl restart dnsmasq.service

In [9]:
sudo /etc/init.d/dnsmasq status

● dnsmasq.service - dnsmasq - A lightweight DHCP and caching DNS server
   Loaded: loaded (/lib/systemd/system/dnsmasq.service; disabled; vendor preset: enabled)
   Active: active (running) since Mo 2018-02-19 21:01:09 UTC; 3min 53s ago
  Process: 5549 ExecStartPost=/etc/init.d/dnsmasq systemd-start-resolvconf (code=exited, status=0/SUCCESS)
  Process: 5530 ExecStart=/etc/init.d/dnsmasq systemd-exec (code=exited, status=0/SUCCESS)
  Process: 5527 ExecStartPre=/usr/sbin/dnsmasq --test (code=exited, status=0/SUCCESS)
 Main PID: 5548 (dnsmasq)
    Tasks: 1
   Memory: 552.0K
      CPU: 55ms
   CGroup: /system.slice/dnsmasq.service
           └─5548 /usr/sbin/dnsmasq -x /var/run/dnsmasq/dnsmasq.pid -u dnsmas...

Feb 19 21:01:08 rdf-dev systemd[1]: Starting dnsmasq - A lightweight DHCP a.....
Feb 19 21:01:08 rdf-dev dnsmasq[5527]: dnsmasq: Syntaxprüfung OK.
Feb 19 21:01:09 rdf-dev dnsmasq[5548]: gestartet, Version 2.75, Cachegröße 150
Feb 19 21:01:09 rdf-dev dnsmasq[5548]: Übersetzungsoption

In [11]:
cat /etc/hosts

127.0.0.1	localhost
127.0.1.1	rdf-dev.vm	rdf-dev

# The following lines are desirable for IPv6 capable hosts
::1     localhost ip6-localhost ip6-loopback
ff02::1 ip6-allnodes
ff02::2 ip6-allrouters


In [12]:
dig rdf-dev.vm


; <<>> DiG 9.10.3-P4-Ubuntu <<>> rdf-dev.vm
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 47631
;; flags: qr aa rd ra; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 1

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 1280
;; QUESTION SECTION:
;rdf-dev.vm.			IN	A

;; ANSWER SECTION:
rdf-dev.vm.		0	IN	A	127.0.1.1

;; Query time: 0 msec
;; SERVER: 127.0.0.1#53(127.0.0.1)
;; WHEN: Mon Feb 19 21:06:25 UTC 2018
;; MSG SIZE  rcvd: 55



## Server Konfiguration

- Der eigene FQDN darf in /etc/hosts nicht auf eine lokale Adresse
    verweisen !-)

In [ ]:
#/etc/hosts
127.0.0.1  localhost
192.168.0.1  mars  mars.sol
...

-------

## Konfiguration Family

Muss noch angepasst werden

## Datei

/etc/dnsmasq.conf

In [ ]:
# Schnittstelle zum LAN
interface=eth1
# lokale Hosts nicht dem Upstream-Nameserver melden
domain-needed
bogus-priv
# Domainname sol im LAN
local=/gxy.loc/
domain=gxy.loc
expand-hosts
# Dnsmasq auch für das Gateway (Upstream-Nameserver = 192.168.99.117)
server=192.168.99.117
no-resolv
# dynamische Adressen
dhcp-range=192.168.0.20,192.168.0.50,24h
# statische Adressen
dhcp-host=00:c0:ee:51:39:9f,client03,192.168.0.3,infinite

## Client

Auf DHCP einstellen und ggf. resolv.conf workaround.

-------

## Ergänzung

in /etc/dhcp3/dhclient.conf

In [ ]:
…
send host-name "\<hostname\>";

-------

/etc/network/interfaces

In [ ]:
…
auto eth1
iface eth1 inet dhcp

-------

Ggf. Link /etc/resolv.conf löschen (Wenn kein resolvconf gewünscht)